In [ ]:
import ROOT
import numpy as np
from scipy.constants import c, electron_mass as me
from scipy.constants import elementary_charge as qe, Boltzmann as kb

In [ ]:
def calculate_omega(b_field, charge=-qe, energy=0.0, mass=me):
    """ Calculate cyclotron frequency for a particle in a magnetic field

    Args:
        b_field: Magnetic Field (Tesla)
        charge: Particle charge. Default: electron charge
        energy: Particle kinetic energy (KeV). Default: 0.0
                (yields non-relativistic result)
        mass: Particle mass (Kg). Default: Electron mass
    """

    gamma_m0 = mass + 1e3 * energy * qe / c**2
    return charge * np.array(b_field) / gamma_m0

In [ ]:
def analytic_solution(x0, y0, u0, v0, e0, time):
    omega = calculate_omega(1.0, energy=e0)
    
    # Phase
    phase = time * omega + np.arctan2(u0, v0)
    # Magnitude of velocity, shouldn't change
    vmag = np.sqrt(u0**2 + v0**2)
    
    u1 = np.sin(phase) * vmag
    v1 = np.cos(phase) * vmag
    
    x1 = x0 - v1 / omega
    y1 = y0 + u1 / omega
    print('Analytic solution = ', x1, y1)

In [ ]:
def process_results(fname="build/qtnm.root"):
    inputFile = ROOT.TFile(fname)
    itree = inputFile.Get('Setup')
    otree = inputFile.Get('Watch')
    # Could check here that the number of entries is the same
    for i in range(itree.GetEntries()):
        itree.GetEntry(i)
        otree.GetEntry(i)
        # Particle initial positions
        x0 = itree.Posx
        y0 = itree.Posy
        z0 = itree.Posz
        # Kinetic energy
        e0 = itree.Kine * qe * 1e3 # Joules
        gamma0 = e0 / me / c**2 + 1
        p0 = np.sqrt((e0 + me * c**2)**2 - (me * c**2)**2) / c

        # Initial momenta
        px0 = itree.Px * p0
        py0 = itree.Py * p0
        pz0 = itree.Pz * p0
        
        # Initial velocities
        vx0 = px0 / me / gamma0
        vy0 = py0 / me / gamma0
        vz0 = pz0 / me / gamma0
                
        # Particle end position and energy
        x1 = otree.Posx
        y1 = otree.Posy
        z1 = otree.Posz
        e1 = otree.Kine * qe * 1e3
        gamma1 = e1 / me / c**2 + 1
        
        # Calculate momentum
        p1 = np.sqrt((e1 + me * c**2)**2 - (me * c**2)**2) / c
        px1 = otree.Px * p1
        py1 = otree.Py * p1
        pz1 = otree.Pz * p1

        
        # End time
        t1 = otree.ExitTime * 1e-6

        # Get analytic solution
        e0_kev = e0 / qe / 1e3
        analytic_solution(x0, y0, vx0, vy0, e0_kev, t1)
        print('Numerical solution = ', x1, y1)

In [ ]:
process_results()